In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace
# Initialize Spark session
spark = SparkSession.builder \
    .appName("Jobs") \
    .getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1746654698499_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
s3_path = "s3://aws-emr-studio-201107706822-us-east-1/1746053900403/e-6HYCE3WANA8IROUS7NBVDSOU4/jobs_csv/window_5.csv"
output_base_path = "s3://aws-emr-studio-201107706822-us-east-1/1746053900403/e-6HYCE3WANA8IROUS7NBVDSOU4/processed/windows/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df = spark.read.csv(s3_path, header=True, inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------+--------------------+--------------------+--------------------+-------------+-----+-------+-----+--------------------+-------------------+
|JobID|WindowID|               Title|         Description|        Requirements|         City|State|Country| Zip5|           StartDate|            EndDate|
+-----+--------+--------------------+--------------------+--------------------+-------------+-----+-------+-----+--------------------+-------------------+
|  146|       5|                  RN|<b>    <b>RN</b> ...|We are searching ...|     Longwood|   FL|     US|32779|2012-04-27 13:06:...|2012-05-26 23:59:59|
|  148|       5|   Vacation Planners|DISCOVER DREAM JO...|Please refer to t...|      Orlando|   FL|     US| NULL|2012-04-28 09:01:...|2012-05-27 23:59:59|
|  149|       5|       PHARMACY MGR.|<b>    <b>PHARMAC...|Please refer to t...|      Orlando|   FL|     US| NULL|2012-04-29 05:09:...|2012-05-28 23:59:59|
|  151|       5|             PA/ARNP|PA/ARNP - BusyDer...|Please refer

In [5]:
columns_to_drop = ["City", "Country", "Zip5", "StartDate", "EndDate"]
df = df.drop(*columns_to_drop)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Column that contains HTML
html_column = "description"  # Replace with your actual column name

# Remove HTML tags using regex
df = df.withColumn(html_column, regexp_replace(html_column, "<[^>]*>", ""))

# Show cleaned data
df.select(html_column).show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:
df = df.withColumn(html_column, regexp_replace(html_column, "[\\n\\r\\t]", " "))

from pyspark.sql.functions import trim
df = df.withColumn(html_column, trim(df[html_column]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------+--------------------+--------------------+--------------------+-----+
|JobID|WindowID|               Title|         description|        Requirements|State|
+-----+--------+--------------------+--------------------+--------------------+-----+
|  146|       5|                  RN|RN   —— FT/PT 3p ...|We are searching ...|   FL|
|  148|       5|   Vacation Planners|DISCOVER DREAM JO...|Please refer to t...|   FL|
|  149|       5|       PHARMACY MGR.|PHARMACY MGR.    ...|Please refer to t...|   FL|
|  151|       5|             PA/ARNP|PA/ARNP - BusyDer...|Please refer to t...|   FL|
|  158|       5|   ACTIVITY DIRECTOR|"    ACTIVITY DIR...|Please refer to t...|   FL|
|  162|       5|Clinician/Supervisor|CLINICIAN/ SUPERV...|Please refer to t...|   FL|
|  164|       5|      Credit Analyst|Credit Analyst Gr...|Please refer to t...|   FL|
|  170|       5|  RESERVATION AGENTS|RESERVATION AGENT...|Please refer to t...|   FL|
|  171|       5|       SALES MANAGER|SALES MANAGER - E

In [9]:
# Combine Title and Description
from pyspark.sql.functions import concat_ws, col, split
df = df.withColumn("combined_text", concat_ws(" ", col("Title"), col("Description")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# Tokenize the combined text
df = df.withColumn("tokens", split(col("combined_text"), "\\s+"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
from pyspark.ml.feature import Word2Vec
word2vec = Word2Vec(vectorSize=100, minCount=2, inputCol="tokens", outputCol="job_vector")
model = word2vec.fit(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+
|JobID|WindowID|               Title|         description|        Requirements|State|       combined_text|              tokens|
+-----+--------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+
|  146|       5|                  RN|RN   —— FT/PT 3p ...|We are searching ...|   FL|RN RN   —— FT/PT ...|[RN, RN, ——, FT/P...|
|  148|       5|   Vacation Planners|DISCOVER DREAM JO...|Please refer to t...|   FL|Vacation Planners...|[Vacation, Planne...|
|  149|       5|       PHARMACY MGR.|PHARMACY MGR.    ...|Please refer to t...|   FL|PHARMACY MGR. PHA...|[PHARMACY, MGR., ...|
|  151|       5|             PA/ARNP|PA/ARNP - BusyDer...|Please refer to t...|   FL|PA/ARNP PA/ARNP -...|[PA/ARNP, PA/ARNP...|
|  158|       5|   ACTIVITY DIRECTOR|"    ACTIVITY DIR...|Please refer to t...|   FL|ACTIVITY DIRECTOR..

In [ ]:
# Transform the DataFrame
vectorized_df = model.transform(df).select("JobID", "WindowID", "job_vector")

In [ ]:
vectorized_df.show()

In [ ]:
# Write to S3, partitioned by window
output_path = f"{output_base_path}window={7}/"
vectorized_df.write.mode("overwrite").parquet(output_path)